In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# Step 1: Load and Preprocess Data
# Load data from CSV
csv_file_path = 'hand_gesture_data.csv'
data = pd.read_csv(csv_file_path)

In [3]:
# Separate features and labels
X = data.drop('label', axis=1).values
y = data['label'].values

In [4]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [5]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Step 2: Build and Train the Model
# Define the model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation='softmax')
])

In [7]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Train the model
history = model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test))

Epoch 1/500
14/14 [==============================] - 3s 45ms/step - loss: 2.1038 - accuracy: 0.2801 - val_loss: 1.9872 - val_accuracy: 0.4815
Epoch 2/500
14/14 [==============================] - 0s 10ms/step - loss: 1.8795 - accuracy: 0.5000 - val_loss: 1.7463 - val_accuracy: 0.6481
Epoch 3/500
14/14 [==============================] - 0s 10ms/step - loss: 1.6166 - accuracy: 0.5648 - val_loss: 1.4618 - val_accuracy: 0.6944
Epoch 4/500
14/14 [==============================] - 0s 11ms/step - loss: 1.3547 - accuracy: 0.6343 - val_loss: 1.1754 - val_accuracy: 0.7315
Epoch 5/500
14/14 [==============================] - 0s 10ms/step - loss: 1.0947 - accuracy: 0.6713 - val_loss: 0.9539 - val_accuracy: 0.7130
Epoch 6/500
14/14 [==============================] - 0s 11ms/step - loss: 0.9270 - accuracy: 0.7106 - val_loss: 0.7808 - val_accuracy: 0.7963
Epoch 7/500
14/14 [==============================] - 0s 12ms/step - loss: 0.8143 - accuracy: 0.7361 - val_loss: 0.6871 - val_accuracy: 0.8148
Epoch 

In [9]:
# Step 3: Save the Model in HDF5 Format
hdf5_model_path = 'hand_gesture_model.h5'
model.save(hdf5_model_path)
print(f"Model saved to {hdf5_model_path}")

Model saved to hand_gesture_model.h5


C:\Users\thoma\anaconda3\envs\mediapipe\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Step 4: Convert and Save the Model in TFLite Format
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\thoma\AppData\Local\Temp\tmpwxuevnw_\assets


INFO:tensorflow:Assets written to: C:\Users\thoma\AppData\Local\Temp\tmpwxuevnw_\assets


In [11]:
# Save the TFLite model
tflite_model_path = 'hand_gesture_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
print(f"TFLite model saved to {tflite_model_path}")

TFLite model saved to hand_gesture_model.tflite


In [12]:
# Optional: Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

4/4 - 0s - loss: 0.0083 - accuracy: 0.9907 - 57ms/epoch - 14ms/step

Test accuracy: 0.9907407164573669


In [13]:
# quantize tflite
import tensorflow as tf
import numpy as np
import pandas as pd

# Load your pre-trained Keras model
model = tf.keras.models.load_model('hand_gesture_model.h5')

# Load your dataset
csv_file_path = 'hand_gesture_data.csv'
data = pd.read_csv(csv_file_path)
X = data.drop('label', axis=1).values

# Representative dataset generator
def representative_data_gen():
    for i in range(100):
        input_data = X[i].reshape(1, -1).astype(np.float32)
        yield [input_data]

# Convert the model to a TFLite model with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert and save the quantized model
tflite_model_quant = converter.convert()
with open('hand_gesture_model_quantized.tflite', 'wb') as f:
    f.write(tflite_model_quant)

print("Model quantization complete and saved as 'hand_gesture_model_quantized.tflite'")


INFO:tensorflow:Assets written to: C:\Users\thoma\AppData\Local\Temp\tmphby_jd5j\assets


INFO:tensorflow:Assets written to: C:\Users\thoma\AppData\Local\Temp\tmphby_jd5j\assets
C:\Users\thoma\anaconda3\envs\mediapipe\Lib\site-packages\tensorflow\lite\python\convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Model quantization complete and saved as 'hand_gesture_model_quantized.tflite'
